In [1]:
import scanpy as sc
import pandas as pd 
import numpy as np
import sys
import matplotlib.pyplot as plt
import gc
import anndata
import glob
from multiprocessing import Pool
import os
from sklearn import metrics


/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-p

# AIFI Labels

In [2]:
files_list = glob.glob(os.path.join('/home/workspace/private/bri_figure_all_files_test/jupyter/Onek1k_dataset_followup/Labels_AIFI/', 
                                    "*L1_predicted_labels.csv"))
AIFI_L1 = pd.concat((pd.read_csv(f) for f in files_list), ignore_index=True)

files_list = glob.glob(os.path.join('/home/workspace/private/bri_figure_all_files_test/jupyter/Onek1k_dataset_followup/Labels_AIFI/', 
                                    "*L2_predicted_labels.csv"))
AIFI_L2 = pd.concat((pd.read_csv(f) for f in files_list), ignore_index=True)

files_list = glob.glob(os.path.join('/home/workspace/private/bri_figure_all_files_test/jupyter/Onek1k_dataset_followup/Labels_AIFI/', 
                                    "*L3_predicted_labels.csv"))
AIFI_L3 = pd.concat((pd.read_csv(f) for f in files_list), ignore_index=True)

In [4]:
AIFI_L3

,Unnamed: 0,barcode,predicted_labels
0,0,AAACCTGAGTACCGGA-13,GZMB- CD27+ EM CD4 T cell
1,1,AAACCTGCAAAGCAAT-13,CM CD4 T cell
2,2,AAACGGGAGTTAAGTG-13,GZMB- CD27- EM CD4 T cell
3,3,AAACGGGCAGCCTGTG-13,Adaptive NK cell
4,4,AAACGGGGTCTCGTTC-13,Core naive CD4 T cell
...,...,...,...
1248975,1044,TTTGGTTCAAGTCTAC-68,Core naive CD4 T cell
1248976,1045,TTTGGTTGTGAGGCTA-68,GZMK+ CD27+ EM CD8 T cell
1248977,1046,TTTGTCAAGTCGAGTG-68,Core naive CD4 T cell
1248978,1047,TTTGTCATCACTTACT-68,CD56bright NK cell


In [3]:
Origional_Labels_L1=pd.read_csv('/home/jupyter/Maxim_dataset_followup/GEX_HTO_processed/all_pbmcs/all_pbmcs_metadata.csv')
Origional_Labels_L1=Origional_Labels_L1[["Unnamed: 0","Cluster_names"]]

In [4]:
files_list = glob.glob('/home/jupyter/Maxim_dataset_followup/GEX_HTO_processed/**/*meta*.csv', recursive=True)

In [5]:
files_list = [item for item in files_list if item != '/home/jupyter/Maxim_dataset_followup/GEX_HTO_processed/all_pbmcs/all_pbmcs_metadata.csv']
proj_meta=pd.read_csv(files_list[0])
proj_meta["Cluster_names"]='progenitor'
proj_meta.to_csv(files_list[0])

In [6]:
Origional_Labels_L2 = pd.concat(
    (pd.read_csv(f).assign(filename=os.path.splitext(os.path.basename(f))[0]) for f in files_list),
    ignore_index=True
)


In [7]:
Origional_Labels_L2['filename'] = Origional_Labels_L2['filename'].str.replace("_metadata", "", regex=False)

In [8]:
Origional_Labels_L2['filename'] = Origional_Labels_L2['filename'].str.replace("_", " ", regex=False)

In [9]:
Origional_Labels_L2["Cluster_names"]= Origional_Labels_L2["Cluster_names"].astype(str)+" "+Origional_Labels_L2["filename"].astype(str)

In [10]:
Origional_Labels_L2=Origional_Labels_L2[["Unnamed: 0","Cluster_names"]]

# rename columns

In [5]:
overlapping_barcodes=AIFI_L1['barcode'].tolist()

In [6]:
AIFI_L1.index= AIFI_L1['barcode'].tolist()
AIFI_L2.index= AIFI_L2['barcode'].tolist()
AIFI_L3.index= AIFI_L3['barcode'].tolist()
AIFI_L1=AIFI_L1.loc[list(overlapping_barcodes)]
AIFI_L2=AIFI_L2.loc[list(overlapping_barcodes)]
AIFI_L3=AIFI_L3.loc[list(overlapping_barcodes)]

AIFI_L1=AIFI_L1[['predicted_labels']]
AIFI_L2=AIFI_L2[['predicted_labels']]
AIFI_L3=AIFI_L3[['predicted_labels']]

AIFI_L1.columns= ['AIFI_L1']
AIFI_L2.columns= ['AIFI_L2']
AIFI_L3.columns= ['AIFI_L3']

# Compare

In [7]:
merged_df = pd.concat([AIFI_L1, AIFI_L2, AIFI_L3], axis=1)

In [8]:
merged_df.to_csv("labels.csv")